<a href="https://colab.research.google.com/github/hdpark1208/StudyCode/blob/main/11)_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***5-3 트리의 앙상블***

# **줄거리**

1.  앙상블 학습(Ensemble learning) 중 대표적인 알고리즘인 랜덤 포레스트를 학습한다  

2.  와인 데이터를 불러온 뒤 인풋,타겟 데이터를 추출해서 훈련/테스트세트로 나눈다  

3.  랜덤 포레스트 객체를 만들고 교차 검증을 사용해 결과를 확인한다  

4.  랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도를 출력한다  

5.  램덤 포레스트의 OOB(out of bag) 기능을 이용한다(교차 검증의 효과)  

6.  램덤 포레스트와의 차이를 살피며 엑스트라 트리를 학습한다  
(부트스트랩 샘플을 사용하지않는다 / 노드를 무작위로 분할한다)   

7.  그레이디언트 부스팅, 히스토그램 기반 그레이디언트 부스팅을 학습한다  


#Note

*  (비)정형데이터, 데이터베이스, (NO)SQL
* 부트스트랩 방식 : 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식
* 앙상블 학습 : 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘
* 랜덤 포레스트 : 결정 트리 기반의 앙상블 학습 방법 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만든다
* 엑스트라 트리 : 부트스트랩 샘플을 사용하지않고 랜덤하게 노드를 분할한다(과대적합을 방지할 수 있다)
* 그레이디언트 부스팅 : 앞의 알고리즘들과 다르게 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법. 훈련속도가 느리지만 좋은 성능을 기대할 수 있다.  
더 발전시킨 것이 히스토그램 기반 그레이디언트 부스팅(입력 특성을 256개의 구간으로 나눈다)

#**코드**


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)    #CPU 코어 모두 사용
scores = cross_validate(rf,train_input,train_target,return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)    #특성 중요도 출력

print([0.12345626,0.86862934,0.0079144])    #결정 트리에서의 특성 중요도

#차이나는 이유 : 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문. 그 결과 하나의 특성에 과도하게 집중하지않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다 이는 일반화 성능을 높이는데 도움이 됨


[0.23167441 0.50039841 0.26792718]
[0.12345626, 0.86862934, 0.0079144]


In [ ]:
rf =RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)    #OOB(out of bag : 부트스트랩 샘플에 포함되지않고 남는 샘플)으로 평가한 점수. 교차검증 대신 사용가능

0.8934000384837406


In [ ]:
from sklearn.ensemble import ExtraTreesClassifier    #결정 트리는 최적의 분할을 찾는데 시간을 많이 소모. 특히 고려해야할 특성의 개수가 많을 때. 하지만 이건 램덤하게 노드를 분할하기 때문에 빠르다
et = ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores = cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))    

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input,train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state= 42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)    #결정 트리 개수를 500개로 5배 늘리고 학습률은 2배 높혔다. 과대적합을 잘 억제하고있다.
scores = cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))


0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input,train_target)    #랜덤 포레스트와 비교해서 일부 특성(여기선 당도)에 더 집중한다
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [ ]:
!pip install sklearn.ensemble


ERROR: Could not find a version that satisfies the requirement sklearn.ensemble (from versions: none)
ERROR: No matching distribution found for sklearn.ensemble


In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state= 42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))


0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input,train_target)
result = permutation_importance(hgb,train_input,train_target,
                                n_repeats=10, random_state=42,n_jobs=-1)
print(result.importances_mean)    #특성 중요도, 평균, 표준 편차

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb,test_input,test_target,    #테스트세트에서의 결과
                                n_repeats=10, random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input,test_target)    #결과확인. 2절의 랜덤서치에서는 86%였다 상승함

0.8723076923076923

In [ ]:
from xgboost import XGBClassifier    #사이킷런 외의 히스토그램기반 그레이디언트 부스팅 알고리즘을 구현한 라이브러리 XGBoost,LightGBM
xgb = XGBClassifier(tree_method='hist',random_state=42)
scores = cross_validate(xgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb,train_input,train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284
0.9338079582727165 0.8789710890649293
